In [23]:
# import block
import pandas as pd
import numpy as np
import pyomo.environ as pyo
import matplotlib.pyplot as plt

In [24]:
# load raw train data
# parse data
data = None
try:
    with open('data/DebrisFlow.txt', 'r') as f:
        data = f.read()
except:
    print('Error reading file')
if data is not None:
    # split data into lines
    data = data.split('\n')
    # remove empty lines
    data = [line for line in data if line != '']
    # split lines into columns
    data = [line.split('\t') for line in data]
    # convert to numpy array
    data = np.array(data, dtype=np.float64)

In [ ]:
# function to plot the model results
def plot_model_results(data: np.ndarray, c_res: list, d_res: list, delta_res: list = None):
    '''
     Plots the model results against the data

     IMPORTANT
     ---------
     provide c, d and delta values in a list even if there is only one value

     Parameters
     ==========
          - data: the data to plot
          - c_res: the c results (slope)
          - d_res: the d results (intercept)
          - delta_res: the delta results (segment number for each data point)

     Returns
     =======
     None
    '''

    fig = plt.figure(figsize=(10, 5))
    if delta_res is None:
      plt.plot(data[:,0], data[:,1], 'k.')
      plt.plot(data[:,0], [c_res[0]*data[i,0]+d_res[0] for i in range(data.shape[0])])
    else:
      colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
      marker_options = ['o', 'v', '^', 's', 'p', '*', '+', 'x']
      segment_list = np.unique(delta_res)
      data_length = data.shape[0]

      y_pred = [[c_res[b]*data[i,0]+d_res[b] for i in range(data_length)] for b in segment_list]
      x_segmented = {b:[] for b in segment_list}
      y_true_segmented = {b:[] for b in segment_list}
      y_pred_segmented = {b:[] for b in segment_list}

      for i in range(data_length):
        segment = delta_res[i]
        x_segmented[segment].append(data[i,0])
        y_true_segmented[segment].append(data[i,1])
        y_pred_segmented[segment].append(c_res[segment]*data[i,0] + d_res[segment])

      # plot the model results
      # plot the data
      for segment in x_segmented.keys():
          plt.plot(x_segmented[segment],y_true_segmented[segment], color='k',linewidth=0, marker=marker_options[segment], markerfacecolor='none')
      for y in y_pred:
          plt.plot(data[:,0],y, linestyle='--', linewidth=0.5, color='k')
      # plot the model results
      for segment in x_segmented.keys():
          plt.plot(x_segmented[segment], y_pred_segmented[segment],color = colors[segment], linestyle='-', linewidth=1,
                  label=f'Model Results for segment {segment}')

    plt.xlabel('Distance (m)')
    plt.ylabel('Debris Flow (m)')
    # set the legend
    plt.legend()
    # show the plot
    plt.show()